<a href="https://colab.research.google.com/github/broisdead/autoencoder-/blob/main/program_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
23# denoising_autoencoder_mnist.py
"""
Denoising Autoencoder for MNIST (Keras/TensorFlow)
- Adds Gaussian noise to MNIST images
- Trains a Conv Autoencoder to reconstruct clean images
- Evaluates using MSE and PSNR
- Saves model to /mnt/data/denoising_autoencoder_mnist.h5 (or current directory)
"""

import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt

# Import Keras / TF (ensure installed)
from tensorflow import keras
from tensorflow.keras import layers

# ---------- Parameters ----------
noise_factor = 0.5
epochs = 10
batch_size = 128
n_examples_to_display = 6
model_save_path = "denoising_autoencoder_mnist.h5"

# ---------- Load and preprocess MNIST ----------
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32")  / 255.0
x_train = np.expand_dims(x_train, -1)  # (N,28,28,1)
x_test  = np.expand_dims(x_test, -1)

# ---------- Create noisy inputs ----------
rng = np.random.RandomState(42)
x_train_noisy = x_train + noise_factor * rng.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy  = x_test  + noise_factor * rng.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy  = np.clip(x_test_noisy, 0., 1.)

# ---------- Build convolutional autoencoder ----------
input_img = keras.Input(shape=(28,28,1))

# Encoder
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2,2), padding='same')(x)
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2,2), padding='same')(x)

# Decoder
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2,2))(x)
decoded = layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

# ---------- Train ----------
history = autoencoder.fit(
    x_train_noisy, x_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(x_test_noisy, x_test),
    verbose=2
)

# ---------- Reconstruct and Evaluate ----------
reconstructions = autoencoder.predict(x_test_noisy, batch_size=256)
mse_per_image = np.mean(np.square(reconstructions - x_test), axis=(1,2,3))
mse_mean = float(np.mean(mse_per_image))
# PSNR per-image (images in [0,1]) ; add small eps to avoid log(0)
psnr_per_image = 20.0 * np.log10(1.0) - 10.0 * np.log10(mse_per_image + 1e-12)
psnr_mean = float(np.mean(psnr_per_image))

# Show metrics in a small table (printed)
metrics = pd.DataFrame({
    "metric": ["MSE (test mean)", "PSNR (dB, test mean)"],
    "value": [mse_mean, psnr_mean]
})
print("\nEvaluation metrics:\n", metrics.to_string(index=False))

# ---------- Create a single image showing original | noisy | reconstructed for several samples ----------
def make_grid_image(origs, noisys, recons):
    # Each row: original | noisy | reconstructed
    rows = []
    n = min(len(origs), len(noisys), len(recons))
    for i in range(n):
        orig = (origs[i].squeeze() * 255).astype(np.uint8)
        noisy = (noisys[i].squeeze() * 255).astype(np.uint8)
        recon = (recons[i].squeeze() * 255).astype(np.uint8)
        row = np.concatenate([orig, noisy, recon], axis=1)
        rows.append(row)
    grid = np.concatenate(rows, axis=0)
    return grid

indices = np.arange(n_examples_to_display)
grid = make_grid_image(x_test[indices], x_test_noisy[indices], reconstructions[indices])

plt.figure(figsize=(6, 2*n_examples_to_display))
plt.axis('off')
plt.imshow(grid, cmap='gray', vmin=0, vmax=255)
plt.title("Each row: original | noisy | reconstructed")
plt.show()

# ---------- Save model ----------
autoencoder.save(model_save_path)
print(f"Saved model to: {os.path.abspath(model_save_path)}")

# ---------- Optional: Save a few reconstructed images to disk ----------
import imageio
out_image_path = "recon_grid_mnist.png"
imageio.imwrite(out_image_path, grid)
print(f"Saved reconstruction grid to: {os.path.abspath(out_image_path)}")
